In [1]:
#%install_ext autotime.py
%load_ext autotime

The autotime module is not an IPython extension.


In [2]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import os
from tensorflow.examples.tutorials.mnist import input_data

# Xavier init is well explained here in Andy's blog:
# http://andyljones.tumblr.com/post/110998971763/an-explanation-of-xavier-initialization
#
# Another good explanation can be found in Prateek Joshi's blog:
# https://prateekvjoshi.com/2016/03/29/understanding-xavier-initialization-in-deep-neural-networks/
def xavier_init(size):
    input_dim = size[0]
    xavier_variance = 1. / tf.sqrt(input_dim/2.)
    return tf.random_normal(shape=size, stddev=xavier_variance)

In [4]:
#latent values in generation
OUTPUT_POSSIBILITIES = 100 #100

#audio window
INPUT_DIMENSIONS = 4410 #4410 #6616 10757 #784

#number of hidden nodes?
HIDDEN_NODES = 1000 # 128

#number of training iterations
TRAINING_ITERATIONS = 100000

In [16]:
#abs
#dataGAN = [np.abs(x.ys[:4410]) for x in testingset]#
segment1 = np.random.rand(10757,)
print(len(segment1))


testingset = [
              np.abs(segment1),
             ]

dataGAN = [np.abs(x) for x in testingset]

10757


In [17]:
Z = tf.placeholder(tf.float32, shape=[None, OUTPUT_POSSIBILITIES], name='Z')

G_W1 = tf.Variable(xavier_init([OUTPUT_POSSIBILITIES, HIDDEN_NODES]), name='G_W1')
G_b1 = tf.Variable(tf.zeros(shape=[HIDDEN_NODES]), name='G_b1')

G_W2 = tf.Variable(xavier_init([HIDDEN_NODES, INPUT_DIMENSIONS]), name='G_W2')
G_b2 = tf.Variable(tf.zeros(shape=[INPUT_DIMENSIONS]), name='G_b2')

theta_G = [G_W1, G_W2, G_b1, G_b2]

In [18]:
X = tf.placeholder(tf.float32, shape=[None, INPUT_DIMENSIONS], name='X')

#original-----
D_W1 = tf.Variable(xavier_init([INPUT_DIMENSIONS, HIDDEN_NODES]), name='D_W1')
D_b1 = tf.Variable(tf.zeros(shape=[HIDDEN_NODES]), name='D_b1')

D_W2 = tf.Variable(xavier_init([HIDDEN_NODES, 1]), name='D_W2')
D_b2 = tf.Variable(tf.zeros(shape=[1]), name='D_b2')

theta_D = [D_W1, D_W2, D_b1, D_b2]

In [19]:
def generator(z):
    G_h1 = tf.nn.relu(tf.matmul(z, G_W1) + G_b1)
    G_log_prob = tf.matmul(G_h1, G_W2) + G_b2
    G_prob = tf.nn.sigmoid(G_log_prob)

    return G_prob

def discriminator(x):
    D_h1 = tf.nn.relu(tf.matmul(x, D_W1) + D_b1)
    D_logit = tf.matmul(D_h1, D_W2) + D_b2
    D_prob = tf.nn.sigmoid(D_logit)

    return D_prob, D_logit

G_sample = generator(Z)

D_real, D_logit_real = discriminator(X)
D_fake, D_logit_fake = discriminator(G_sample)

# Loss functions from the paper
# D_loss = -tf.reduce_mean(tf.log(D_real) + tf.log(1. - D_fake))
# G_loss = -tf.reduce_mean(tf.log(D_fake))

# Alternative loss functions
D_loss_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_real, labels=tf.ones_like(D_logit_real)))
D_loss_fake = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_fake, labels=tf.zeros_like(D_logit_fake)))
D_loss = D_loss_real + D_loss_fake
G_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_fake, labels=tf.ones_like(D_logit_fake)))

# Update D(X)'s parameters
D_solver = tf.train.AdamOptimizer().minimize(D_loss, var_list=theta_D)

# Update G(Z)'s parameters
G_solver = tf.train.AdamOptimizer().minimize(G_loss, var_list=theta_G)

def sample_Z(m, n):
    return np.random.uniform(-1., 1., size=[m, n])

batch_size = 100 #128
Z_dim = OUTPUT_POSSIBILITIES

sess = tf.Session()
sess.run(tf.global_variables_initializer())

#mnist = input_data.read_data_sets('MNIST/', one_hot=True)

In [20]:
#np.shape(mnist.train.next_batch(10)[0])

In [21]:
#np.shape(mnist.train.next_batch(10)[1])

In [ ]:
if not os.path.exists('output/'):
    os.makedirs('output/')

i = 0
sampleHolder = []

for itr in range(TRAINING_ITERATIONS):
    if itr % 10000 == 0:
        
        numSamples = 1
        samples = sess.run(G_sample, feed_dict={Z: sample_Z(numSamples, Z_dim)})
        
        #fig = plot(samples)
        #plt.savefig('output/{}.png'.format(str(i).zfill(3)), bbox_inches='tight')
        #i += 1
        #plt.close(fig)
        
        ## construct wave from sampling during training
        #post_reconstructions = ffttowave(   samples   ) # if using fft, transform back first?
        sampleHolder.append(samples)


    #CHANGHING INPUTS    
    #X_mb, _ = mnist.train.next_batch(batch_size)
    #X_mb = [dataGAN[0] for x in range(0,batch_size)]
    X_mb = [ dataGAN[np.random.choice([i for i in range(0,len(dataGAN))],1)[0]] for x in range(0,batch_size)]
    
    #X_mb = [transforms[0]]

    _, D_loss_curr = sess.run([D_solver, D_loss], feed_dict={X: X_mb, Z: sample_Z(batch_size, Z_dim)})
    _, G_loss_curr = sess.run([G_solver, G_loss], feed_dict={Z: sample_Z(batch_size, Z_dim)})

    if itr % 1000 == 0:
        print('Iter: {}'.format(itr))
        print('D loss: {:.4}'. format(D_loss_curr))
        print('G_loss: {:.4}'.format(G_loss_curr))
        print()